In [1]:
import math
import numpy as np

In [2]:
#Parâmetro: Um dicionário, contendo como chave o código do documento e como valor o texto "bruto" do documento, 
    #sem formatações, 
#Retorno: Um dicionário, contendo como chave o código do documento e como valor o texto formatado do documento,
    #sem acentuações, caracteres especiais e letras que não estão no Padrão ASCII, com todas as letras são minúsculas e
    #separadas por espaço " " e armazenadas em um vetor
def formataString (s):
    import unicodedata
    
    s = ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
    
    s = s.lower()
    
    caracteresEspeciais = ['!', '\"', '#', '$', '%', '&', '\'', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>',
                           '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
    for carac in caracteresEspeciais:
        if (s.count(carac) > 0):
            s = s.replace(carac, "")
        
    strings = s.split()
    
    return strings

In [3]:
def indiceInvertido (arquivos):
    vocabulario = []

    for k, v in arquivos.items():
        #print(k, " : ", v)

        ignorarIndice = []
        for i in range(len(v)):
            frequenciaNesteDoc = 1
            if (ignorarIndice.count(i) != 0):
                continue

            for j in range(i+1, len(v)):
                if (v[i] == v[j]):
                    frequenciaNesteDoc+=1
                    ignorarIndice.append(j)

            vocabulario.append([v[i], k, frequenciaNesteDoc])

    #print("Vocabulário:")
    #for val in vocabulario:
    #    print(val[0], " -> ", val[1], " : ", val[2])
    #print()
    
    ############################################################ TODO: Separar essas funções?
    
    indInv = {}

    for val in vocabulario:
        if val[0] in indInv.keys():
            dados = indInv[val[0]]
            dados.append([val[1], val[2]])
            indInv[val[0]] = dados
        else:
            dados = [[val[1],val[2]]]
            indInv[val[0]] = dados

    #print("A palavra em questão -> [Os documentos em que ela aparece, a quantidade total que ela aparece nesse doc]")
    #for k, v in indInv.items():
    #    print(k, "-> ", v)
        
    #############################################################
    
    """for k, v in indInv.items():
        nx = len(v[0])
        idf = math.log(N/nx)
        v.append(idf)
        indInv[k] = v"""
        
    return indInv

In [4]:
def coordenadas (indInv, numDocs):
    coords = np.zeros((len(indInv), numDocs))
    indicePalavra = 0;
    
    for k, v in indInv.items():
        nx = len(v)
        idf = math.log(numDocs/nx)
        #print(idf)
        for dados in v:
            w = dados[1] * idf
            coords[indicePalavra][dados[0]] = w
        
        indicePalavra+=1
        
    return coords

In [5]:
def formataQuery (indInv, indQ, numDocs):
    Q = np.zeros((len(indInv),1))
    indicePalavra = 0;
    
    for k, v in indInv.items():
        if k in indQ.keys():
            nx = len(v)
            idf = math.log(numDocs/nx)
            
            w = (indQ[k])[0][1] * idf
            Q[indicePalavra] = w
        indicePalavra+=1
    return Q

In [20]:
def verificaConsulta(coords, Q):
    resultado = {}
    Q = Q.reshape((len(Q),1))
    
    for i in range(np.shape(coords)[1]):
        vec = coords[:, i]
        vec = vec.reshape((len(vec),1))
        similaridade = (vec.T @ Q)/(np.linalg.norm(vec) * np.linalg.norm(Q))
        resultado[i] = similaridade[0][0]
    return resultado

In [24]:
txt0 = "Bátátá? Tentarei,comêndo umas 12 mais rápida-mente. ]]} Çentido!!!"
txt1 = "Alô?? Veio me ver!! Que demóóóóóraaa!!!"
txt2 = "Sem-pre tentando fazer. O certo nunca alcança"
txt3 = "ASSIM VERÁ: nunca me achará, escondido estarei-me"
txt4 = "137 = cento e trinta e seis + um"
txt5 = "Como disse? \"Vou-me embora daquí!!!! \'Mané\'\"."
txt6 = "teste tentando doze cabeca andarilho triplo triplo"
txt7 = "batata tentativa nove triplo quando falar batata teste"

arquivos = {0:txt0, 1:txt1, 2:txt2, 3:txt3, 4:txt4, 5:txt5, 6:txt6, 7:txt7}
###IMPORTANTE: As chaves devem sempre começar do 0

"""txt0 = "A A A B"
txt1 = "A A C"
txt2 = "A A"
txt3 = "B B"

arquivos = {0:txt0, 1:txt1, 2:txt2, 3:txt3}"""

for k, v in arquivos.items():
    arquivos[k] = formataString(v)

#for k, v in arquivos.items():
#    print(v)
    
indInv = indiceInvertido(arquivos)

#print("A palavra em questão -> [O documento em que ela aparece, a quantidade total que ela aparece nesse doc], ")
#for k, v in indInv.items():
#    print(k, "-> ", v)

coords = coordenadas(indInv, len(arquivos))

In [28]:
consulta = "tentando a tentativa"
query = {0:formataString(consulta)}
indQ = indiceInvertido(query)

#print(indiceQ)

Q = formataQuery(indInv, indQ, len(arquivos))

#print(Q)

resultado = verificaConsulta(coords, Q)
print("Maior valor = maior prioridade:")
print()
for k, v in resultado.items():
    print("Valor para o documento ", k, " : ", v)

Maior valor = maior prioridade:

Valor para o documento  0  :  0.0
Valor para o documento  1  :  0.0
Valor para o documento  2  :  0.1523878635518919
Valor para o documento  3  :  0.0
Valor para o documento  4  :  0.0
Valor para o documento  5  :  0.0
Valor para o documento  6  :  0.1553471274761235
Valor para o documento  7  :  0.32225169331774484
